### Basic testing of NALU

See [here](https://arxiv.org/abs/1808.00508) for more info on the theory behind Neural Arithmetic Logic units.

This notebook is loosely based on the implementation found [here](https://github.com/grananqvist/NALU-tf).



In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from nac import nac
from nalu import nalu
import GenData as gd

In [ ]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [ ]:
epochs = 100
learning_rate = 0.01 # Per author's comment
batch_size = 20

X_data, Y_data = gd.gd_uniform(size=10000)

X = tf.placeholder(tf.float32, shape=[None, 2]) 
Y_true = tf.placeholder(tf.float32, shape=[None, 1])
Y_pred = nalu(X, 1)

loss = tf.losses.absolute_difference(Y_pred, Y_true) # testing with l1
optimizer = tf.train.RMSPropOptimizer(learning_rate) # Per author's comment
train_op = optimizer.minimize(loss)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
nac_summary = tf.summary.scalar('nac', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [ ]:
for ep in range(epochs):    
    i = 0
    gts = 0
        
    while i < len(X_data):
        xs, ys = X_data[i:i+batch_size], Y_data[i:i+batch_size]
        _, ys_pred, l = sess.run([train_op, Y_pred, loss], feed_dict={X: xs, Y_true: ys})

            # calculate number of correct predictions from batch
        gts += np.sum(np.isclose(ys, ys_pred, atol=1e-4, rtol=1e-4)) 
        
        # Log it for tensorBoard.
        summary_str = nac_summary.eval(session=sess, feed_dict={X: xs, Y_true: ys})
        step = ep *  len(X_data) + i
        file_writer.add_summary(summary_str, step)
        
        i += batch_size
        
        
    acc = gts/len(Y_data)
    print('epoch {2}, loss: {0}, accuracy: {1}'.format(l, acc, ep))

In [ ]:
#saver = tf.train.Saver()
#save_path = saver.save(sess, "./model.ckpt")

In [ ]:
file_writer.close()

In [ ]:
import pandas as pd
plot_data = pd.DataFrame(data={'index': [], 'error':[]})

In [ ]:
for i in range(20, 5000):
    Xtest, Ytest = gen_data_unif.generate_dataset(_min=i*10, _max=(i+1)*10-1, size=500)
    _, _, l = sess.run([train_op, Y_pred, loss], feed_dict={X: Xtest, Y_true: Ytest})
    plot_data = pd.concat([plot_data, pd.DataFrame({'index':[i*10], 'error':[l]})])

In [ ]:
plot_data.plot(x='index', y='error')